# Inverter

## Introduction

"Inverter" refers to the common image processing operation of inversion. This is not to be confused with the logical inverter. 

## Working of the IP

- The Inverter IP is an AXI Streaming IP with 1 slave and 1 master. 
- This takes in 32bit data and splits the 32 bit data into 8 bit chunks.
- It subtracts the 1 byte from 255 to obtain the inverse of the data
- This means that each for an input pixel (which is usually 8 bit) we get the inverse of that pixel 

## Programming the Board with the bitstream

- RPHAX generates the `.bit` `.hwh` and `.tcl` files. Ensure that the three files have the same name and place them in any directory inside the `jupyter_notebooks`  directory. (You can place them anywhere in PYNQ) 

- We program the bitstream onto the PL(FPGA Part of the Zynq) within python using the Overlay class inside the pynq class
- PYNQ Framework will interpret the hardware design from the `.hwh` - Hardware Handoff file and then program the Bitstream onto the FPGA
- Previous versions of PYNQ used the `.tcl` file of the block design, but later on hardware handoff was used when the classes were upgraded

In [58]:
from pynq import Overlay

ol = Overlay("./overlay/design_1.bit")
#ol2 = Overlay("./overlay/dma_tutorial.bit")

- You get a quick overview of the contents of the overlay
    - Hierarichy
    - IPs
    - Methods and attributes available

In [48]:
ol?

In [59]:
# ol.axi_dma_0.device


/usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/overlay.py:681: UserWarning: Interrupt mm2s_introut not created: Could not find UIO device for interrupt pin for IRQ number 62
  warnings.warn('Interrupt {} not created: {}'.format(


### Interacting with the AXI DMA

- Note that the Processing System will be able to interact only with the DMA Directly, and our inverter cannot be "seen" as an IP by the Processor, since we are using a streaming IP and the processor has only AXI4 interfaces
- Data is sent to the DMA which internally has the capabality to convert Memory Mapped interface to Streaming Interface which can be then used to connect to the inverter. 
- In the below block, we just refer the objects by shorter identifiers for convenience 

In [60]:
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

- The data is sent as through the shared buffer. `allocate` is used to send interface with the shared buffer
Note: The Latest version of pynq (version 2.7), where as previously `Xlnk` was used for the same
- To convert any old overlay to the latest image do the following
    - Replace `from pynq import Xlnk` with `from pynq import allocate`
    - Replace 
    ```
    xlnk = Xlnk()
    in_buffer = xlnk.cma_array(shape=(data_size,), dtype=np.uint32)
    ```
    with 
    ```
    in_buffer = allocate(shape=(data_size,), dtype=np.uint32)
    ```

In [61]:
from pynq import allocate
import numpy as np

data_size = 1000
input_buffer = allocate(shape=(data_size,), dtype=np.uint32)

### Populate the data to be sent into the IP

In [62]:
for i in range(data_size):
    input_buffer[i] = i + 0x00000001

In [63]:
for i in range(10):
    print(hex(input_buffer[i]))

0x1
0x2
0x3
0x4
0x5
0x6
0x7
0x8
0x9
0xa


### Send the data into the DMA

In [64]:
dma_send.transfer(input_buffer)


### Initialize the output buffer

In [65]:
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


### Receive the data from the output buffer

In [66]:
dma_recv.transfer(output_buffer)


In [67]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

0xfffffffe
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


**Note:** Here, we receive only `255-first_value` as the RTL has to be modified with suiable AXIS signals to indicate end of a packet and beginning of the next packet

## Additional references on building overlays
    [Pynq Overlay Tutorial](https://pynq.readthedocs.io/en/latest/overlay_design_methodology/overlay_tutorial.html)